### Step 1: Load Time-Series Data from S3

We will begin by loading the BLS time-series dataset (`pr.data.0.Current`) stored in our S3 bucket. This dataset contains quarterly data for various `series_id` codes, along with their respective `year`, `period`, and `value`.

We'll use `boto3` to connect to AWS S3 and `pandas` to load and process the data.


In [47]:
import boto3
import pandas as pd
import io

# Initialize S3 client
s3 = boto3.client('s3')
bucket_name = 'rearc-data-raw'

# Define the S3 key (file path inside the bucket)
ts_key = 'bls/pr.data.0.Current'

# Fetch the object from S3
ts_obj = s3.get_object(Bucket=bucket_name, Key=ts_key)

# Read the file content into a Pandas DataFrame
ts_df = pd.read_csv(io.BytesIO(ts_obj['Body'].read()), delimiter='\t')

# Clean column names and values
ts_df.columns = ts_df.columns.str.strip()  # Remove extra spaces from column headers
ts_df["series_id"] = ts_df["series_id"].str.strip()  # Clean series_id values
ts_df["period"] = ts_df["period"].str.strip()        # Clean period values

# Show first few rows of the data
print("Cleaned Sample Data")
ts_df.head()


Cleaned Sample Data


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


Part 2: Load Population JSON Data from S3

### Step 2: Load Population Data from S3

This step loads the population data (`population.json`) from S3. The data comes from the public API at `datausa.io`, which we earlier fetched and stored in the `population/` folder.

We will normalize the nested JSON structure into a flat pandas DataFrame using `json_normalize()`.


In [48]:
import json

# Key for the population JSON file
pop_key = 'population/population.json'

# Fetch the file from S3
pop_obj = s3.get_object(Bucket=bucket_name, Key=pop_key)
pop_data = json.loads(pop_obj['Body'].read())

# Normalize nested JSON to a flat table
pop_df = pd.json_normalize(pop_data, record_path='data')

# Display the first few rows
pop_df.head()


,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2023,2023,332387540,united-states
1,01000US,United States,2022,2022,331097593,united-states
2,01000US,United States,2021,2021,329725481,united-states
3,01000US,United States,2020,2020,326569308,united-states
4,01000US,United States,2019,2019,324697795,united-states


Part 3a: Population Statistics (2013–2018)
We’ll calculate:

Mean of the U.S. population from 2013 to 2018 (inclusive)

Standard deviation of the same

### Step 3a: US Population Statistics (2013–2018)

Now that we've loaded the population data, let's filter it for the years 2013 through 2018 and calculate:
- The **mean** population
- The **standard deviation** of the population

This helps us understand overall population trends during that period.


In [49]:
# Ensure Year column is in numeric format
pop_df["Year"] = pop_df["Year"].astype(int)

# Filter for years between 2013 and 2018 (inclusive)
filtered_pop = pop_df[(pop_df["Year"] >= 2013) & (pop_df["Year"] <= 2018)]

# Calculate population statistics
mean_population = filtered_pop["Population"].mean()
std_population = filtered_pop["Population"].std()

print("US Population Mean (2013–2018):", int(mean_population))
print("US Population Standard Deviation (2013–2018):", int(std_population))


US Population Mean (2013–2018): 317437383
US Population Standard Deviation (2013–2018): 4257089


Part 3b: Best Year by Total Quarterly Value (Time-Series Data)
We’ll identify:

The year in which each series_id had the highest total "value" (i.e., the sum of all quarters for that year)

### Step 3b: Best Year by Series ID (Based on Total Value)

Using the BLS time-series dataset (`pr.data.0.Current`), we want to identify the year that had the highest summed value for each unique `series_id`.

This helps us detect when a series performed its best over time.


In [50]:
# Ensure correct data types
ts_df["year"] = ts_df["year"].astype(int)
ts_df["value"] = pd.to_numeric(ts_df["value"], errors="coerce")

# Drop rows with invalid or missing values
ts_df = ts_df.dropna(subset=["value"])

# Group by series_id and year, then calculate the total value per year
grouped = ts_df.groupby(["series_id", "year"])["value"].sum().reset_index()

# Find the year with the maximum value per series_id
best_year_df = grouped.loc[grouped.groupby("series_id")["value"].idxmax()].reset_index(drop=True)

# Display result
print("Top 5 series with their best year based on total value:")
best_year_df.head()


Top 5 series with their best year based on total value:


,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400


In [54]:
print(pop_df.columns)

Index(['ID Nation', 'Nation', 'ID Year', 'year', 'Population', 'Slug Nation'], dtype='object')


In [56]:
#Before merging in Step 3c, make sure both sides are aligned:
# Clean column names
pop_df.columns = pop_df.columns.str.strip()

# Ensure year is integer
pop_df["year"] = pop_df["year"].astype(int)

# Optional: preview cleaned population data
print(pop_df[["year", "Population"]].head())


   year  Population
0  2023   332387540
1  2022   331097593
2  2021   329725481
3  2020   326569308
4  2019   324697795


In [60]:
# Step: Filter the time-series data to only years that exist in population data
valid_years = pop_df["year"].unique()
filtered_ts = filtered_ts[filtered_ts["year"].isin(valid_years)]

# Step: Merge population data
final_merged = filtered_ts.merge(pop_df[["year", "Population"]], on="year", how="left")

# View final output
print("Final Joined Report (Series ID PRS30006032, Q01):")
final_merged.head()


Final Joined Report (Series ID PRS30006032, Q01):


,series_id,year,period,value,footnote_codes,Population
0,PRS30006032,2013,Q01,0.5,NaN,311536594
1,PRS30006032,2014,Q01,-0.1,NaN,314107084
2,PRS30006032,2015,Q01,-1.7,NaN,316515021
3,PRS30006032,2016,Q01,-1.4,NaN,318558162
4,PRS30006032,2017,Q01,0.9,NaN,321004407


Step 3c: Join Time-Series & Population Data
We will:

Filter the time-series data to only include:

series_id = "PRS30006032"

period = "Q01"

Join it with the population dataset on the year column

Output the series_id, year, period, value, and Population

### Step 3c: Join Time-Series with Population Data

Now, we’ll join the two datasets:
- Filter the BLS dataset to get `series_id = PRS30006032` and `period = Q01`
- Join this with the US population dataset by matching the `year`
This helps us analyze how the value for this series aligns with population trends over time.


In [ ]:
# Merge population into time series data for PRS30006032 and Q01
final_merged = filtered_ts.merge(pop_df[["year", "Population"]], on="year", how="left")

# Show final result
print("Final Joined Report (Series ID PRS30006032, Q01):")
final_merged.head()

Final Joined Report (Series ID PRS30006032, Q01):


,series_id,year,period,value,footnote_codes,Population
0,PRS30006032,2013,Q01,0.5,NaN,311536594
1,PRS30006032,2014,Q01,-0.1,NaN,314107084
2,PRS30006032,2015,Q01,-1.7,NaN,316515021
3,PRS30006032,2016,Q01,-1.4,NaN,318558162
4,PRS30006032,2017,Q01,0.9,NaN,321004407


Step 3c: Final Joined Report (Time-Series + Population)
In this step, we filtered the BLS time-series dataset to extract values for:

series_id = PRS30006032

period = Q01 (Quarter 1)

Years that exist in our population dataset (2013–2023)

We then joined this filtered dataset with the population data on the year column.

This gave us a combined view of the time-series metric and the corresponding US population for each year.

This merged dataset will help in trend analysis of economic indicators against population growth.